In [1]:
import numpy as np
import pandas as pd

import re

In [2]:
df_inbox = pd.read_json (r'.\data\inbox.json')
df_inbox.head()

,owner,from,to,cc,date,subject,message_id,text,forward
0,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:51:45+00:00,RE: IRPM-202 отсутствует инструкция по установ...,<a1562077a5ff40249595581be759d4f5@omega.sbrf.ru>,"Просьба приложить это в виде инструкции (""рыбу...",[{'text': 'Просьба приложить это в виде инстру...
1,Gaparkhoev.V.L@omega.sbrf.ru,Gaparkhoev.V.L@omega.sbrf.ru,"[Konyrev.D.An@sberbank.ru, Fedotov.I.Ser@omega...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:39:55+00:00,Re: IRPM-202 отсутствует инструкция по установ...,<1577785193.3767.0.camel@omega.sbrf.ru>,"Добрый день, Илья!\r\nМодели и квоты не меняли...","[{'text': 'Добрый день, Илья! Модели и квоты н..."
2,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[AVSipiev@sberbank.ru, Bagnenko.V.V@sberbank.r...",2019-12-31 09:11:08+00:00,IRPM-202 отсутствует инструкция по установке н...,<37eaa2b19f3249bcb211646d4a1a9bdc@omega.sbrf.ru>,"Коллеги, добрый день!\r\n\r\n\r\n\r\nПри устан...","[{'text': 'Коллеги, добрый день! При установке..."
3,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-30 14:31:10+00:00,[ACL Portal] Newsletter 2019,<e310dfa11be393364b4403f537f38d52@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,[[ВНЕШНЯЯ ПОЧТА: Если вы не знаете от...
4,Ustinov.A.Yury@omega.sbrf.ru,Ustinov.A.Yury@omega.sbrf.ru,[Smirnov.I.Yurye@omega.sbrf.ru],"[Konyrev.D.An@sberbank.ru, Tsutsiev.D.E@omega....",2019-12-30 13:28:29+00:00,RE: Построение модели извлечения,<960e189c9be64d4b80e9bc2cb84a6523@omega.sbrf.ru>,"Иван, добрый вечер!\r\n\r\nХотелось бы уточнит...","[{'text': 'Иван, добрый вечер! Хотелось бы уто..."


In [3]:
df_inbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3843 entries, 0 to 3842
Data columns (total 9 columns):
owner         3843 non-null object
from          3843 non-null object
to            3843 non-null object
cc            3843 non-null object
date          3843 non-null datetime64[ns, UTC]
subject       3806 non-null object
message_id    3843 non-null object
text          3843 non-null object
forward       3827 non-null object
dtypes: datetime64[ns, UTC](1), object(8)
memory usage: 270.3+ KB


In [4]:
df_inbox['owner'].value_counts()

moy_sberbank@sberbank.ru                394
Yusupov.V.A@omega.sbrf.ru               379
Shtuntsayger.I.Ef@sberbank.ru           182
Burlakova.M.Val@sberbank.ru             173
MYUKudryavtcev@sberbank.ru              173
                                       ... 
obuchenie_razvitie_pcphr@sberbank.ru      1
DKSudnev@sberbank.ru                      1
info@sberbank-team.com                    1
Emelyanov.A.Aleksan@sberbank.ru           1
greendays@sberbank-birthday.com           1
Name: owner, Length: 262, dtype: int64

In [5]:
df_inbox['owner'].nunique()

262

In [6]:
df_outbox = pd.read_json (r'.\data\outbox.json')
df_outbox.head()

,owner,from,to,cc,date,subject,message_id,text,forward
0,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Ocheretny.A.S@sberbank.ru],[],2019-12-27 11:09:47+00:00,FW: DS - итоги 2019,<f41b3f2d974c4117be1ec9139b6f779d@sberbank.ru>,"Андрей, привет!\r\n\r\nСделано, смотри.\r\n\r\...","[{'text': 'Андрей, привет! Сделано, смотри. С ..."
1,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],"[MYUKudryavtcev@sberbank.ru, Shavrina.T.O@sber...",2019-12-27 08:11:06+00:00,RE: LJ-образец.sql,<ad52878dc4f945d79c18bbe135026f86@sberbank.ru>,"Андрей, привет!\r\n\r\nДовольно забавный форма...","[{'text': 'Андрей, привет! Довольно забавный ф..."
2,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],[],2019-12-24 14:23:51+00:00,Сервер с GPU для Башен,<c4229bc0b1784748a426bad0d08177c1@sberbank.ru>,"Андрей, привет!\r\n\r\nВ продолжении разговора...","[[Андрей, привет!\nВ продолжении разговора про..."
3,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-12-24 12:59:47+00:00,Роли в команде,<08b6ad91178543f7b4aada2eab27fe1f@sberbank.ru>,"Добрый день,\r\n\r\nКак и договаривались высыл...","[[Добрый день,\nКак и договаривались высылаю р..."
4,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Burlakova.M.Val@sberbank.ru, Konyrev.D.An@ome...",[Protopopov.V.Al@omega.sbrf.ru],2019-12-24 12:46:18+00:00,RE: Оценка правовых рисков,<f317433a695740f1b1d54c71e04307bc@sberbank.ru>,"Добрый день!\r\n\r\nСпасибо, мы сейчас с коман...","[{'text': 'Добрый день! Спасибо, мы сейчас с к..."


In [7]:
set(df_inbox['message_id']) & set(df_outbox['message_id'])

{'<7e5f52332ea04b14ad61a1b07b42db9d@sberbank.ru>'}

In [8]:
list(df_inbox[df_inbox['message_id']=='<7e5f52332ea04b14ad61a1b07b42db9d@sberbank.ru>']['text'])

['Коллеги, добрый день.\r\n\r\n\r\n\r\nДефект, который появился в пн был устранен, сборка поставлена на ИФТ.\r\n\r\n\r\n\r\nhttp://sbrf-nexus.ca.sbrf.ru/nexus/content/repositories/Nexus_PROD/Nexus_PROD/CI02029517_ML_CDS/cds/01.001.00-55/cds-01.001.00-55-distrib.zip\r\n\r\n\r\n\r\nПросьба поставить сборку на ПРОМ.\r\n\r\n\r\n\r\nС уважением,\r\n\r\nКонырев Дмитрий\r\n\r\n']

In [9]:
list(df_outbox[df_outbox['message_id']=='<7e5f52332ea04b14ad61a1b07b42db9d@sberbank.ru>']['text'])

['Коллеги, добрый день.\r\n\r\n\r\n\r\nДефект, который появился в пн был устранен, сборка поставлена на ИФТ.\r\n\r\n\r\n\r\nhttp://sbrf-nexus.ca.sbrf.ru/nexus/content/repositories/Nexus_PROD/Nexus_PROD/CI02029517_ML_CDS/cds/01.001.00-55/cds-01.001.00-55-distrib.zip\r\n\r\n\r\n\r\nПросьба поставить сборку на ПРОМ.\r\n\r\n\r\n\r\nС уважением,\r\n\r\nКонырев Дмитрий\r\n\r\n']

In [10]:
df_outbox['owner'].value_counts()

Konyrev.D.An@sberbank.ru    768
Name: owner, dtype: int64

In [11]:
df_inbox[df_inbox['from']=='Konyrev.D.An@sberbank.ru']

,owner,from,to,cc,date,subject,message_id,text,forward
1070,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Bagnenko.V.V@sberbank.ru, Alekseev.D.Genna@om...","[Timofeev.A.Valery@omega.sbrf.ru, Shtuntsayger...",2019-10-09 13:33:12+00:00,Устранение дефекта сервисов CDS ТД,<7e5f52332ea04b14ad61a1b07b42db9d@sberbank.ru>,"Коллеги, добрый день.\r\n\r\n\r\n\r\nДефект, к...","[[Коллеги, добрый день.\nДефект, который появи..."


In [12]:
df_outbox[df_outbox['owner']=='Konyrev.D.An@sberbank.ru']

,owner,from,to,cc,date,subject,message_id,text,forward
0,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Ocheretny.A.S@sberbank.ru],[],2019-12-27 11:09:47+00:00,FW: DS - итоги 2019,<f41b3f2d974c4117be1ec9139b6f779d@sberbank.ru>,"Андрей, привет!\r\n\r\nСделано, смотри.\r\n\r\...","[{'text': 'Андрей, привет! Сделано, смотри. С ..."
1,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],"[MYUKudryavtcev@sberbank.ru, Shavrina.T.O@sber...",2019-12-27 08:11:06+00:00,RE: LJ-образец.sql,<ad52878dc4f945d79c18bbe135026f86@sberbank.ru>,"Андрей, привет!\r\n\r\nДовольно забавный форма...","[{'text': 'Андрей, привет! Довольно забавный ф..."
2,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],[],2019-12-24 14:23:51+00:00,Сервер с GPU для Башен,<c4229bc0b1784748a426bad0d08177c1@sberbank.ru>,"Андрей, привет!\r\n\r\nВ продолжении разговора...","[[Андрей, привет!\nВ продолжении разговора про..."
3,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-12-24 12:59:47+00:00,Роли в команде,<08b6ad91178543f7b4aada2eab27fe1f@sberbank.ru>,"Добрый день,\r\n\r\nКак и договаривались высыл...","[[Добрый день,\nКак и договаривались высылаю р..."
4,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Burlakova.M.Val@sberbank.ru, Konyrev.D.An@ome...",[Protopopov.V.Al@omega.sbrf.ru],2019-12-24 12:46:18+00:00,RE: Оценка правовых рисков,<f317433a695740f1b1d54c71e04307bc@sberbank.ru>,"Добрый день!\r\n\r\nСпасибо, мы сейчас с коман...","[{'text': 'Добрый день! Спасибо, мы сейчас с к..."
...,...,...,...,...,...,...,...,...,...
763,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Karpets.M.Val@omega.sbrf.ru],[],2019-01-11 11:32:07+00:00,None,<075292f91f9d4e6998d6d7165326a40a@CAB-VSP-MBX1...,\r\n,"[[\n, 0, {'text': ' '}]]"
764,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Karpets.M.Val@omega.sbrf.ru],[],2019-01-10 12:27:40+00:00,None,<2a9cb95ffc234def93861d5306c5f12c@CAB-VSP-MBX1...,\r\n,"[[\n, 0, {'text': ' '}]]"
765,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[OUT-Dmitrieva-AA@mail.ca.sbrf.ru],[],2019-01-10 08:47:57+00:00,FW: Наиболее актуальная версия Глоссария,<05b523ac07214903b809e33b10c8c9fe@CAB-VSP-MBX1...,\r\n\r\nFrom: Бурлакова Марина Валериевна <Bur...,"[{'text': ' '}, {'to': ['Конырев Дмитрий Андре..."
766,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Protopopov.V.Al@omega.sbrf.ru, Protopopov.V.A...",[],2019-01-09 15:38:41+00:00,Последний отчет,<8a434a7eae854b8590d39bfc7d1883ce@CAB-VSP-MBX1...,"Добрый день,\r\n\r\nВысылаю в письме отчет по ...","[[Добрый день,\nВысылаю в письме отчет по сущн..."


In [13]:
df_outbox['to'].value_counts()

[Konyrev.D.A@mail.ca.sbrf.ru]                                                                                                                                           66
[MYUKudryavtcev@sberbank.ru]                                                                                                                                            60
[Yusupov.V.A@omega.sbrf.ru]                                                                                                                                             35
[Burlakova.M.Val@sberbank.ru]                                                                                                                                           25
[Burlakova.M.Val@sberbank.ru, Konyrev.D.An@omega.sbrf.ru]                                                                                                               21
                                                                                                                                                 

In [100]:
def search_pattern(pattern, input_to):
    #pattern = "Konyrev.D.An@sberbank.ru"
    Flag = False
    
        
    if type(input_to)==str:
        lst = re.findall(pattern, input_to)
        if len(lst)>0:
            Flag=True
    elif input_to is not None:
        for input_txt in input_to:
            lst = re.findall(pattern, input_txt)
            if len(lst)>0:
                Flag=True
                break

    return Flag

In [15]:
list(df_inbox['to'])

[['Gaparkhoev.V.L@omega.sbrf.ru', 'Konyrev.D.An@sberbank.ru'],
 ['Konyrev.D.An@sberbank.ru', 'Fedotov.I.Ser@omega.sbrf.ru'],
 ['Gaparkhoev.V.L@omega.sbrf.ru', 'Konyrev.D.An@sberbank.ru'],
 ['Konyrev.D.An@sberbank.ru'],
 ['Smirnov.I.Yurye@omega.sbrf.ru'],
 ['Ulyanov.A.A@sberbank.ru',
  'Ocheretny.A.S@sberbank.ru',
  'NRZelinskiy@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Shavrina.T.O@sberbank.ru'],
 ['Ocheretny.A.S@sberbank.ru',
  'NRZelinskiy@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Krivobok.M.A@sberbank.ru',
  'Shavrina.T.O@sberbank.ru'],
 ['moy_sberbank@sberbank.ru'],
 ['NRZelinskiy@sberbank.ru',
  'Ocheretny.A.S@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Krivobok.M.A@sberbank.ru'],
 ['Ocheretny.A.S@sberbank.ru',
  'Ulyanov.A.A@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Krivobok.M.A@sberbank.ru'],
 ['AVStrelnikov@sberbank.ru',
  'Shtuntsayger.I.Ef@sberbank.ru',
  'AVS

In [16]:
df_inbox['Konyrev'] = np.vectorize(search_pattern)('Konyrev.D.An@sberbank.ru', df_inbox['to'])

In [17]:
Konyrev_inbox = df_inbox[df_inbox['Konyrev']==True]

In [18]:
Konyrev_inbox.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2189 entries, 0 to 3842
Data columns (total 10 columns):
owner         2189 non-null object
from          2189 non-null object
to            2189 non-null object
cc            2189 non-null object
date          2189 non-null datetime64[ns, UTC]
subject       2153 non-null object
message_id    2189 non-null object
text          2189 non-null object
forward       2177 non-null object
Konyrev       2189 non-null bool
dtypes: bool(1), datetime64[ns, UTC](1), object(8)
memory usage: 173.2+ KB


In [19]:
Konyrev_inbox.head()

,owner,from,to,cc,date,subject,message_id,text,forward,Konyrev
0,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:51:45+00:00,RE: IRPM-202 отсутствует инструкция по установ...,<a1562077a5ff40249595581be759d4f5@omega.sbrf.ru>,"Просьба приложить это в виде инструкции (""рыбу...",[{'text': 'Просьба приложить это в виде инстру...,True
1,Gaparkhoev.V.L@omega.sbrf.ru,Gaparkhoev.V.L@omega.sbrf.ru,"[Konyrev.D.An@sberbank.ru, Fedotov.I.Ser@omega...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:39:55+00:00,Re: IRPM-202 отсутствует инструкция по установ...,<1577785193.3767.0.camel@omega.sbrf.ru>,"Добрый день, Илья!\r\nМодели и квоты не меняли...","[{'text': 'Добрый день, Илья! Модели и квоты н...",True
2,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[AVSipiev@sberbank.ru, Bagnenko.V.V@sberbank.r...",2019-12-31 09:11:08+00:00,IRPM-202 отсутствует инструкция по установке н...,<37eaa2b19f3249bcb211646d4a1a9bdc@omega.sbrf.ru>,"Коллеги, добрый день!\r\n\r\n\r\n\r\nПри устан...","[{'text': 'Коллеги, добрый день! При установке...",True
3,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-30 14:31:10+00:00,[ACL Portal] Newsletter 2019,<e310dfa11be393364b4403f537f38d52@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,[[ВНЕШНЯЯ ПОЧТА: Если вы не знаете от...,True
5,Krivobok.M.A@sberbank.ru,Krivobok.M.A@sberbank.ru,"[Ulyanov.A.A@sberbank.ru, Ocheretny.A.S@sberba...",[AChertok@sberbank.ru],2019-12-30 12:12:04+00:00,RE: материалы со встречи по DS итогам 2019,<67d07ae64f014007b59ab01de9f2c73c@sberbank.ru>,"Коллеги, привет!\r\nОбновлен слайд по TagMe.\r...","[{'text': 'Коллеги, привет! Обновлен слайд по ...",True


In [20]:
df_outbox['Ocheretny'] = np.vectorize(search_pattern)('Ocheretny.A.S@sberbank.ru', df_outbox['to'])
df_outbox.head()

,owner,from,to,cc,date,subject,message_id,text,forward,Ocheretny
0,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Ocheretny.A.S@sberbank.ru],[],2019-12-27 11:09:47+00:00,FW: DS - итоги 2019,<f41b3f2d974c4117be1ec9139b6f779d@sberbank.ru>,"Андрей, привет!\r\n\r\nСделано, смотри.\r\n\r\...","[{'text': 'Андрей, привет! Сделано, смотри. С ...",True
1,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],"[MYUKudryavtcev@sberbank.ru, Shavrina.T.O@sber...",2019-12-27 08:11:06+00:00,RE: LJ-образец.sql,<ad52878dc4f945d79c18bbe135026f86@sberbank.ru>,"Андрей, привет!\r\n\r\nДовольно забавный форма...","[{'text': 'Андрей, привет! Довольно забавный ф...",False
2,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],[],2019-12-24 14:23:51+00:00,Сервер с GPU для Башен,<c4229bc0b1784748a426bad0d08177c1@sberbank.ru>,"Андрей, привет!\r\n\r\nВ продолжении разговора...","[[Андрей, привет!\nВ продолжении разговора про...",False
3,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-12-24 12:59:47+00:00,Роли в команде,<08b6ad91178543f7b4aada2eab27fe1f@sberbank.ru>,"Добрый день,\r\n\r\nКак и договаривались высыл...","[[Добрый день,\nКак и договаривались высылаю р...",True
4,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Burlakova.M.Val@sberbank.ru, Konyrev.D.An@ome...",[Protopopov.V.Al@omega.sbrf.ru],2019-12-24 12:46:18+00:00,RE: Оценка правовых рисков,<f317433a695740f1b1d54c71e04307bc@sberbank.ru>,"Добрый день!\r\n\r\nСпасибо, мы сейчас с коман...","[{'text': 'Добрый день! Спасибо, мы сейчас с к...",False


In [21]:
#Конырев пишет Ocheretny
Ocheretny_outbox = df_outbox[df_outbox['Ocheretny']==True]
Ocheretny_outbox.sort_values(by='date')

,owner,from,to,cc,date,subject,message_id,text,forward,Ocheretny
285,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-09-10 15:36:57+00:00,Примеры ГПХ,<ecb6fb1f16ae4f688158cb498fd57cf7@sberbank.ru>,"Добрый день!\r\n\r\nКак и обещал, высылаю прим...","[[Добрый день!\nКак и обещал, высылаю примеры ...",True
153,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[kondratieva@vbudushee.ru, satikova@vbudushee....",[],2019-10-16 08:05:02+00:00,RE: Выступление на радио Медиаметрикс,<89c7825e5199476f90aad83f398a6753@sberbank.ru>,"Добрый день!\r\n\r\nПо поводу Академии ИИ, дей...","[{'text': 'Добрый день! По поводу Академии ИИ,...",True
151,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[kondratieva@vbudushee.ru, satikova@vbudushee....",[],2019-10-16 12:58:04+00:00,RE: Выступление на радио Медиаметрикс,<f23f496c8fad4f41bbbb32690c83c496@sberbank.ru>,"Добрый день, прошу прощения.\r\n\r\nРуководите...","[{'text': 'Добрый день, прошу прощения. Руково...",True
150,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[kondratieva@vbudushee.ru, satikova@vbudushee....",[],2019-10-16 13:35:38+00:00,RE: Выступление на радио Медиаметрикс,<e0abb98c695a4a9eb441e2772865119b@sberbank.ru>,"К моему сожалению нет.\r\n\r\nС уважением,\r\n...","[{'text': 'К моему сожалению нет. С уважением,...",True
149,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[kondratieva@vbudushee.ru, satikova@vbudushee....",[],2019-10-17 14:39:57+00:00,RE: Выступление на радио Медиаметрикс,<dfc571cdfe5b4186bc6a313758ecc3e0@sberbank.ru>,Добрый день!\r\n\r\nХотелось бы добавить вопро...,[{'text': 'Добрый день! Хотелось бы добавить в...,True
3,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-12-24 12:59:47+00:00,Роли в команде,<08b6ad91178543f7b4aada2eab27fe1f@sberbank.ru>,"Добрый день,\r\n\r\nКак и договаривались высыл...","[[Добрый день,\nКак и договаривались высылаю р...",True
0,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Ocheretny.A.S@sberbank.ru],[],2019-12-27 11:09:47+00:00,FW: DS - итоги 2019,<f41b3f2d974c4117be1ec9139b6f779d@sberbank.ru>,"Андрей, привет!\r\n\r\nСделано, смотри.\r\n\r\...","[{'text': 'Андрей, привет! Сделано, смотри. С ...",True


In [80]:
len(Ocheretny_outbox['forward'].to_list())

7

In [23]:
Ocheretny_outbox.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 285
Data columns (total 10 columns):
owner         7 non-null object
from          7 non-null object
to            7 non-null object
cc            7 non-null object
date          7 non-null datetime64[ns, UTC]
subject       7 non-null object
message_id    7 non-null object
text          7 non-null object
forward       7 non-null object
Ocheretny     7 non-null bool
dtypes: bool(1), datetime64[ns, UTC](1), object(8)
memory usage: 567.0+ bytes


In [24]:
#Ocheretny пишет Коныреву
list(df_inbox[df_inbox['owner']=='Ocheretny.A.S@sberbank.ru']['to'])

[['NRZelinskiy@sberbank.ru',
  'Ulyanov.A.A@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Krivobok.M.A@sberbank.ru'],
 ['Fenogenova.A.S@sberbank.ru',
  'Ulyanov.A.A@sberbank.ru',
  'Shavrina.T.O@sberbank.ru',
  'Krivobok.M.A@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'NRZelinskiy@sberbank.ru'],
 ['Ulyanov.A.A@sberbank.ru',
  'Shavrina.T.O@sberbank.ru',
  'Krivobok.M.A@sberbank.ru',
  'NRZelinskiy@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Fenogenova.A.S@sberbank.ru'],
 ['satikova@vbudushee.ru', 'Konyrev.D.An@sberbank.ru']]

In [25]:
df_inbox[df_inbox['owner']=='Ocheretny.A.S@sberbank.ru'].sort_values(by='date')

,owner,from,to,cc,date,subject,message_id,text,forward,Konyrev
990,Ocheretny.A.S@sberbank.ru,Ocheretny.A.S@sberbank.ru,"[satikova@vbudushee.ru, Konyrev.D.An@sberbank.ru]",[],2019-10-14 14:48:38+00:00,RE: Выступление на радио Медиаметрикс,<278b83c4eda14d5b9f1f2d4f194f0568@sberbank.ru>,"Дария, привет.\r\n\r\nСегодня обсудили с Димо...","[{'text': 'Дария, привет. Сегодня обсудили с Д...",True
24,Ocheretny.A.S@sberbank.ru,Ocheretny.A.S@sberbank.ru,"[Ulyanov.A.A@sberbank.ru, Shavrina.T.O@sberban...",[AChertok@sberbank.ru],2019-12-27 14:06:00+00:00,материалы со встречи по DS итогам 2019,<a129d958fef44e0f9b6e4682a8c700b3@sberbank.ru>,По итогам жду короткое описание от каждой кома...,[[По итогам жду короткое описание от каждой ко...,True
16,Ocheretny.A.S@sberbank.ru,Ocheretny.A.S@sberbank.ru,"[Fenogenova.A.S@sberbank.ru, Ulyanov.A.A@sberb...",[AChertok@sberbank.ru],2019-12-27 21:37:14+00:00,RE: материалы со встречи по DS итогам 2019,<04de50ea70804c3cb7f89c9279a30199@sberbank.ru>,"Коллеги,\r\n\r\nВ приложении сборка, в которую...","[{'text': 'Коллеги, В приложении сборка, в кот...",True
14,Ocheretny.A.S@sberbank.ru,Ocheretny.A.S@sberbank.ru,"[NRZelinskiy@sberbank.ru, Ulyanov.A.A@sberbank...","[AChertok@sberbank.ru, Shavrina.T.O@sberbank.ru]",2019-12-28 15:25:30+00:00,RE: материалы со встречи по DS итогам 2019,<f9a85ed6289c41b8a4d686588689aaa9@sberbank.ru>,"Коллеги, привет.\r\n\r\nВ приложении новая пер...","[{'text': 'Коллеги, привет. В приложении новая...",True


In [86]:
def proba(forward):
    if type(forward)==float:
        return 0
    else:
        return 1

In [87]:
#Konyrev_inbox['forward']
Konyrev_inbox['forward_len'] = Konyrev_inbox['forward'].apply(lambda x : proba(x))
Konyrev_inbox.head()

c:\users\sych_\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,owner,from,to,cc,date,subject,message_id,text,forward,Konyrev,forward_len
0,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:51:45+00:00,RE: IRPM-202 отсутствует инструкция по установ...,<a1562077a5ff40249595581be759d4f5@omega.sbrf.ru>,"Просьба приложить это в виде инструкции (""рыбу...",[{'text': 'Просьба приложить это в виде инстру...,True,1
1,Gaparkhoev.V.L@omega.sbrf.ru,Gaparkhoev.V.L@omega.sbrf.ru,"[Konyrev.D.An@sberbank.ru, Fedotov.I.Ser@omega...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:39:55+00:00,Re: IRPM-202 отсутствует инструкция по установ...,<1577785193.3767.0.camel@omega.sbrf.ru>,"Добрый день, Илья!\r\nМодели и квоты не меняли...","[{'text': 'Добрый день, Илья! Модели и квоты н...",True,1
2,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[AVSipiev@sberbank.ru, Bagnenko.V.V@sberbank.r...",2019-12-31 09:11:08+00:00,IRPM-202 отсутствует инструкция по установке н...,<37eaa2b19f3249bcb211646d4a1a9bdc@omega.sbrf.ru>,"Коллеги, добрый день!\r\n\r\n\r\n\r\nПри устан...","[{'text': 'Коллеги, добрый день! При установке...",True,1
3,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-30 14:31:10+00:00,[ACL Portal] Newsletter 2019,<e310dfa11be393364b4403f537f38d52@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,[[ВНЕШНЯЯ ПОЧТА: Если вы не знаете от...,True,1
5,Krivobok.M.A@sberbank.ru,Krivobok.M.A@sberbank.ru,"[Ulyanov.A.A@sberbank.ru, Ocheretny.A.S@sberba...",[AChertok@sberbank.ru],2019-12-30 12:12:04+00:00,RE: материалы со встречи по DS итогам 2019,<67d07ae64f014007b59ab01de9f2c73c@sberbank.ru>,"Коллеги, привет!\r\nОбновлен слайд по TagMe.\r...","[{'text': 'Коллеги, привет! Обновлен слайд по ...",True,1


In [88]:
Konyrev_inbox[Konyrev_inbox['forward_len']==0]

,owner,from,to,cc,date,subject,message_id,text,forward,Konyrev,forward_len
15,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-28 03:43:21+00:00,Update for Event : COLING Workshop on NLP in E...,<111811a1c3e1fd6c37e85f3d611a7b73@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,NaN,True,0
72,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-22 12:25:58+00:00,Update for Event : COLING Workshop on NLP in E...,<9fb76bf501e9896aaf57cdcea0f178e7@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,NaN,True,0
107,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-19 09:06:09+00:00,Update for Event : SocialNLP 2020@WWW+@ACL Ca...,<3006556e93fe48ef242ee9110af3789e@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,NaN,True,0
113,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-19 01:03:41+00:00,Update for Event : COLING Workshop on NLP in E...,<f0bccb33edd12a030b15fb3dd00d25fd@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,NaN,True,0
270,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-05 16:01:38+00:00,Update for Event : Third Workshop on NLP for I...,<78e4d498327715dde8e2b8dd8b2b22a2@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,NaN,True,0
272,Marakhovskaya.M.S@sberbank.ru,Marakhovskaya.M.S@sberbank.ru,"[Kozlovtseva.E.Ale@sberbank.ru, AChertok@sberb...",[MAEremenko@sberbank.ru],2019-12-05 14:37:10+00:00,FW: Успейте принять участие в опросе вовлечённ...,<1FF5DC2D-C825-413D-B0F9-C08F303FF3D7@sberbank...,"Коллеги,\r\nОчень большая просьба принять учас...",NaN,True,0
294,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-04 17:31:08+00:00,Update for Event : 1st CFP: 15th Workshop on I...,<0b44bac05892642f0f6a79e0b379d991@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,NaN,True,0
581,dmitry.chernous@abbyy.com,dmitry.chernous@abbyy.com,"[Burlakova.M.Val@sberbank.ru, Konyrev.D.An@sbe...","[Protopopov.V.Al@omega.sbrf.ru, Krylova.D.An@o...",2019-11-14 13:03:39+00:00,"RE: Работа с ДКП и Актами ПП, содержащими прил...",<AM6PR02MB5574ACE1BBCC3EF1DFB97718FE710@AM6PR0...,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,NaN,True,0
2161,OUT-Dmitrieva-AA@mail.ca.sbrf.ru,OUT-Dmitrieva-AA@mail.ca.sbrf.ru,[Konyrev.D.An@sberbank.ru],[],2019-07-09 09:24:53+00:00,[Fwd: нормализация атрибутов],<1562664291.6995.0.camel@mail.ca.sbrf.ru>,-------- Пересылаемое сообщение --------\r\nОт...,NaN,True,0
3461,Yusupov.V.A@omega.sbrf.ru,Yusupov.V.A@omega.sbrf.ru,"[Aksenov.S.An@omega.sbrf.ru, Konyrev.D.An@sber...",[Protopopov.V.Al@sberbank.ru],2019-03-04 14:54:12+00:00,RE: Статистика по документам типа Устав,<66c10529d65f4178a42ea8b700afddd7@omega.sbrf.ru>,"Сергей, Дмитрий, добрый день!\r\n\r\nМожем ли ...",NaN,True,0


In [89]:
Konyrev_inbox.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2189 entries, 0 to 3842
Data columns (total 11 columns):
owner          2189 non-null object
from           2189 non-null object
to             2189 non-null object
cc             2189 non-null object
date           2189 non-null datetime64[ns, UTC]
subject        2153 non-null object
message_id     2189 non-null object
text           2189 non-null object
forward        2177 non-null object
Konyrev        2189 non-null bool
forward_len    2189 non-null int64
dtypes: bool(1), datetime64[ns, UTC](1), int64(1), object(8)
memory usage: 190.3+ KB


### Объединим папки Входящие и Исходящие по Коныреву и поищем по теме письма

In [90]:
combine_df = Konyrev_inbox.append(df_outbox, ignore_index = True, sort = False)
combine_df.head()

,owner,from,to,cc,date,subject,message_id,text,forward,Konyrev,forward_len,Ocheretny
0,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:51:45+00:00,RE: IRPM-202 отсутствует инструкция по установ...,<a1562077a5ff40249595581be759d4f5@omega.sbrf.ru>,"Просьба приложить это в виде инструкции (""рыбу...",[{'text': 'Просьба приложить это в виде инстру...,True,1.0,NaN
1,Gaparkhoev.V.L@omega.sbrf.ru,Gaparkhoev.V.L@omega.sbrf.ru,"[Konyrev.D.An@sberbank.ru, Fedotov.I.Ser@omega...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:39:55+00:00,Re: IRPM-202 отсутствует инструкция по установ...,<1577785193.3767.0.camel@omega.sbrf.ru>,"Добрый день, Илья!\r\nМодели и квоты не меняли...","[{'text': 'Добрый день, Илья! Модели и квоты н...",True,1.0,NaN
2,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[AVSipiev@sberbank.ru, Bagnenko.V.V@sberbank.r...",2019-12-31 09:11:08+00:00,IRPM-202 отсутствует инструкция по установке н...,<37eaa2b19f3249bcb211646d4a1a9bdc@omega.sbrf.ru>,"Коллеги, добрый день!\r\n\r\n\r\n\r\nПри устан...","[{'text': 'Коллеги, добрый день! При установке...",True,1.0,NaN
3,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-30 14:31:10+00:00,[ACL Portal] Newsletter 2019,<e310dfa11be393364b4403f537f38d52@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,[[ВНЕШНЯЯ ПОЧТА: Если вы не знаете от...,True,1.0,NaN
4,Krivobok.M.A@sberbank.ru,Krivobok.M.A@sberbank.ru,"[Ulyanov.A.A@sberbank.ru, Ocheretny.A.S@sberba...",[AChertok@sberbank.ru],2019-12-30 12:12:04+00:00,RE: материалы со встречи по DS итогам 2019,<67d07ae64f014007b59ab01de9f2c73c@sberbank.ru>,"Коллеги, привет!\r\nОбновлен слайд по TagMe.\r...","[{'text': 'Коллеги, привет! Обновлен слайд по ...",True,1.0,NaN


In [91]:
combine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2957 entries, 0 to 2956
Data columns (total 12 columns):
owner          2957 non-null object
from           2957 non-null object
to             2957 non-null object
cc             2957 non-null object
date           2957 non-null datetime64[ns, UTC]
subject        2874 non-null object
message_id     2957 non-null object
text           2957 non-null object
forward        2945 non-null object
Konyrev        2189 non-null object
forward_len    2189 non-null float64
Ocheretny      768 non-null object
dtypes: datetime64[ns, UTC](1), float64(1), object(10)
memory usage: 277.3+ KB


In [93]:
len(Konyrev_inbox) + len(df_outbox)

2957

In [103]:
# Выступление на радио Медиаметрикс
combine_df['Subject_Media'] = np.vectorize(search_pattern)('Выступление на радио Медиаметрикс', combine_df['subject'])

In [133]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[12][1].get('clear_text')

'ВНЕШНЯЯ ПОЧТА:\nЕсли вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:\nIf the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nДмитрий, добрый день!\nВысылаю телефон Дарии на всякий случай: +7916566279. Если что, звоните ей\nМожете также выслать свой для оперативной связи?\n11:41, 17 октября 2019 г., "Мария Кондратьева" <kondratieva@vbudushee.ru<mailto:kondratieva@vbudushee.ru>>:\nКоллеги, доброго дня!\nВысылаю список ориентировочный список вопросов интервью. Если хотим осветить еще какие-то моменты, можем добавить свои.\nВопросы, как я уже сказала, ориентировочные, они лишь задают вектор беседы. Но т.к. все будет проходить в формате живого общения, по факту их может стать больше.\n--\n'

In [135]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[12][0]

{'text': 'Добрый день!\nХотелось бы добавить вопрос про тенденции ИИ (общемировые и внутри Банка).\nС уважением,\nКонырев Дмитрий\n'}

## пара входящих писем 

In [136]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[0][1].get('clear_text')

'\nАндрей привет!\n18 октября, 10.00 у нас радиоэфир на Медиаметрикс по нашему профилю. (примерно на час)\nПо плану на этом эфире будет победитель прошлого года, я от лица Академии и очень хотела попросить кого-то от CDS, кто сможет рассказать про задачу и про работу датааналитиков в банке. Можешь помочь найти человека?\n--\n'

In [174]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[0][1]

{'to': [],
 'from': ['"Дария Сатикова"', 'satikova@vbudushee.ru'],
 'date': '2019-08-10T11:13:00',
 'text': '\nАндрей привет!\n18 октября, 10.00 у нас радиоэфир на Медиаметрикс по нашему профилю. (примерно на час)\nПо плану на этом эфире будет победитель прошлого года, я от лица Академии и очень хотела попросить кого-то от CDS, кто сможет рассказать про задачу и про работу датааналитиков в банке. Можешь помочь найти человека?\n--\nС уважением,\nДария Сатикова\nРуководитель программы по цифровым навыкам и компетенциям\nБлаготворительный фонд Сбербанка "Вклад в будущее"\nМоб.тел.: +79165662791\n',
 'message_id': 4636578643699845651,
 'clear_text': '\nАндрей привет!\n18 октября, 10.00 у нас радиоэфир на Медиаметрикс по нашему профилю. (примерно на час)\nПо плану на этом эфире будет победитель прошлого года, я от лица Академии и очень хотела попросить кого-то от CDS, кто сможет рассказать про задачу и про работу датааналитиков в банке. Можешь помочь найти человека?\n--\n',
 'clear_subject'

In [138]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[0][0]

{'text': 'Дария, привет.\nСегодня обсудили с Димой, что ему было бы интересно принять участие. Напиши, пожалуйста, ему подробнее предысторию вопроса.\nС уважением,\nАндрей\n'}

In [155]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['to'])[0]

['satikova@vbudushee.ru', 'Konyrev.D.An@sberbank.ru']

### другая пара входящих

In [140]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[1][1].get('clear_text')

'\nДария, привет.\nСегодня обсудили с Димой, что ему было бы интересно принять участие. Напиши, пожалуйста, ему подробнее предысторию вопроса.\n'

In [142]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[1][0]

{'text': 'ВНЕШНЯЯ ПОЧТА:          Если вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:         If the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nДмитрий, добрый день!\nВ копии моя коллега Мария, она занимается организацией эфира.\nЕсли кратко, то в рамках пиара нашей Олимпиады для школьников по ИИ состоится эфир на Радио Медиаметрикс в 10.00 18 октября (пятница). На нем буду я, как руководитель Академии ИИ, наш победитель Хакатона Академии ИИ прошлого года Артем Михеев и нужен еще эксперт в датааналитике от Сбербанка, чтобы рассказывать про карьеру, знания и будущее ИИ.\nМогу подойти, дать небольшое интро по Академии ИИ, вы же на Кутузовской находитесь?\n--\nС уважением,\nДария Сатикова\nРуководитель программы

In [153]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['to'])[1]

['Ocheretny.A.S@sberbank.ru', 'Konyrev.D.An@sberbank.ru']

In [154]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['cc'])[1]

['kondratieva@vbudushee.ru']

# и ещё

In [157]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[2][1].get('clear_text')

'\nДмитрий, добрый день!\nВ копии моя коллега Мария, она занимается организацией эфира.\nЕсли кратко, то в рамках пиара нашей Олимпиады для школьников по ИИ состоится эфир на Радио Медиаметрикс в 10.00 18 октября (пятница). На нем буду я, как руководитель Академии ИИ, наш победитель Хакатона Академии ИИ прошлого года Артем Михеев и нужен еще эксперт в датааналитике от Сбербанка, чтобы рассказывать про карьеру, знания и будущее ИИ.\nМогу подойти, дать небольшое интро по Академии ИИ, вы же на Кутузовской находитесь?\n--\n'

In [146]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[2][0]

{'text': 'ВНЕШНЯЯ ПОЧТА:          Если вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:         If the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nКоллеги, добрый день!\nМне нужны ваши фотографии для публикации анонса эфира и должность Дмитрия.\nТакже прикладываю памятку от коллег из НТИ:\nПрограмма «Выше среднего» идёт в формате прямого эфира, продолжительность 45 минут.\nНачало эфира 18 октября в 10:00. Просьба приезжать за 15 минут до начала (9:45) по адресу: Кутузовский проспект, 22. Вход с торца здания (в сторону центра) на двери табличка «Клименко и партнеры». Парковки, к сожалению, нет, эвакуаторы работают исправно.\nТрансляция будет идти в эфире на радио и на ютуб-канале, поэтому просьба не приходить в п

In [150]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['to'])[2]

['satikova@vbudushee.ru',
 'Ocheretny.A.S@sberbank.ru',
 'Konyrev.D.An@sberbank.ru']

# и...

In [158]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[3][1].get('clear_text')

'ВНЕШНЯЯ ПОЧТА:\nЕсли вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:\nIf the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nКоллеги, добрый день!\nМне нужны ваши фотографии для публикации анонса эфира и должность Дмитрия.\nТакже прикладываю памятку от коллег из НТИ:\nПрограмма «Выше среднего» идёт в формате прямого эфира, продолжительность 45 минут.\nНачало эфира 18 октября в 10:00. Просьба приезжать за 15 минут до начала (9:45) по адресу: Кутузовский проспект, 22. Вход с торца здания (в сторону центра) на двери табличка «Клименко и партнеры». Парковки, к сожалению, нет, эвакуаторы работают исправно.\nТрансляция будет идти в эфире на радио и на ютуб-канале, поэтому просьба не приходить в полосатой одежде и одежде

In [159]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[3][0]

{'text': 'Добрый день!\nПо поводу Академии ИИ, действительно хотелось бы услышать какие-то вводные. Я сижу на Кутузовском, 11 этаж место 110.\nМоя должность – руководитель направления. Фотографии вышлю до 13:00 текущего дня.\nС уважением,\nКонырев Дмитрий\n'}

In [160]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['to'])[3]

['kondratieva@vbudushee.ru',
 'satikova@vbudushee.ru',
 'Ocheretny.A.S@sberbank.ru']

# и 

In [164]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[4][1].get('clear_text')

'\nДобрый день!\nПо поводу Академии ИИ, действительно хотелось бы услышать какие-то вводные. Я сижу на Кутузовском, 11 этаж место 110.\nМоя должность – руководитель направления. Фотографии вышлю до 13:00 текущего дня.\n'

In [165]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[4][0]

{'text': 'ВНЕШНЯЯ ПОЧТА:          Если вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:         If the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nБольшое спасибо!\nВводные вам вроде бы уже дала Дария. Если будут еще какие-то вопросы, пишите.\nМожно также попросить вас помочь с корректной формулировкой должности для внешней коммуникации? "Руководитель направления" выглядит несколько обрезанной. Какого направления, отдела? Можете прислать развернутую формулировку, чтобы тем, кто сильно вне контекста было хотя бы примерно понятно, чем вы занимаетесь?\n--\nС уважением,\nКондратьева Мария\nСпециалист программы "Цифровые навыки и компетенции"\nБлаготворительный фонд Сбербанка "Вклад в будущее"\nМоб.тел.: +7(925)197-6

In [166]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['to'])[4]

['Konyrev.D.An@sberbank.ru',
 'satikova@vbudushee.ru',
 'Ocheretny.A.S@sberbank.ru']

### hjh

In [167]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[5][1].get('clear_text')

'\nДобрый день!\nПо поводу Академии ИИ, действительно хотелось бы услышать какие-то вводные. Я сижу на Кутузовском, 11 этаж место 110.\nМоя должность – руководитель направления. Фотографии вышлю до 13:00 текущего дня.\n'

In [168]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[5][0]

{'text': 'ВНЕШНЯЯ ПОЧТА:          Если вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:         If the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nДмитрий, простите, что дергаю, но у нас, к сожалению, очень сжатые сроки. Сегодня нужно сделать анонс эфира.\nКогда можно ожидать вашу фотографию и развернутую формулировку должности?\n--\nС уважением,\nКондратьева Мария\nСпециалист программы "Цифровые навыки и компетенции"\nБлаготворительный фонд Сбербанка "Вклад в будущее"\nМоб.тел.: +7(925)197-67-16\n'}

In [172]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['message_id'])[5]

'<7318371571229346@iva4-6593cae50902.qloud-c.yandex.net>'

In [169]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['to'])[5]

['Konyrev.D.An@sberbank.ru',
 'satikova@vbudushee.ru',
 'Ocheretny.A.S@sberbank.ru']

### hjhj

In [170]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[6][1].get('clear_text')

'ВНЕШНЯЯ ПОЧТА:\nЕсли вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:\nIf the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nДмитрий, простите, что дергаю, но у нас, к сожалению, очень сжатые сроки. Сегодня нужно сделать анонс эфира.\nКогда можно ожидать вашу фотографию и развернутую формулировку должности?\n--\n'

In [173]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[6][1]

{'to': ['Конырев Дмитрий Андреевич ',
  'Konyrev.D.An@sberbank.ru',
  ' Дария Сатикова ',
  'satikova@vbudushee.ru',
  ' Очеретный Андрей Сергеевич ',
  'Ocheretny.A.S@sberbank.ru'],
 'from': ['Мария Кондратьева ', 'kondratieva@vbudushee.ru'],
 'date': '2019-10-16T15:36:00',
 'subject': 'Re: Выступление на радио Медиаметрикс',
 'text': 'ВНЕШНЯЯ ПОЧТА:\nЕсли вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:\nIf the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nДмитрий, простите, что дергаю, но у нас, к сожалению, очень сжатые сроки. Сегодня нужно сделать анонс эфира.\nКогда можно ожидать вашу фотографию и развернутую формулировку должности?\n--\nС уважением,\nКондратьева Мария\nСпециалист программы "Цифровые навыки 

In [179]:
for i in range(13):
    m_id = list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[i][1].get('message_id')
    print(m_id)

4636578643699845651
4571527361863631763
3708976331847197209
-6628584906726205050
-6929697904466721554
-6929697904466721554
-1409547540196611424
-8924036844257243765
8347289238976959042
-7199936823807272147
-7199936823807272147
-7199936823807272147
5454167398978121783


In [180]:
combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['message_id']

593        <278b83c4eda14d5b9f1f2d4f194f0568@sberbank.ru>
586     <13191191571127846@myt2-a7d7570d35ff.qloud-c.y...
570     <3626981571211412@iva5-2a2172cb7cff.qloud-c.ya...
2342       <89c7825e5199476f90aad83f398a6753@sberbank.ru>
568     <5367301571219384@myt4-eb6256e01f8b.qloud-c.ya...
566     <7318371571229346@iva4-6593cae50902.qloud-c.ya...
2340       <f23f496c8fad4f41bbbb32690c83c496@sberbank.ru>
565     <7644791571232868@myt5-3a82a06244de.qloud-c.ya...
2339       <e0abb98c695a4a9eb441e2772865119b@sberbank.ru>
564     <7709551571233729@sas2-c8fd3ed78d67.qloud-c.ya...
561     <11899891571301709@iva1-3b1c0a1a240c.qloud-c.y...
556     <2569951571321577@iva3-67f911cb3a01.qloud-c.ya...
2338       <dfc571cdfe5b4186bc6a313758ecc3e0@sberbank.ru>
Name: message_id, dtype: object

In [181]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[9][1].get('clear_text')

'\nК моему сожалению нет.\n'

In [184]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[9][0]

{'text': 'ВНЕШНЯЯ ПОЧТА:          Если вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:         If the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nЯ поняла, спасибо.\n--\nС уважением,\nКондратьева Мария\nСпециалист программы "Цифровые навыки и компетенции"\nБлаготворительный фонд Сбербанка "Вклад в будущее"\nМоб.тел.: +7(925)197-67-16\n'}

In [182]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[10][1].get('clear_text')

'\nК моему сожалению нет.\n'

In [185]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[10][0]

{'text': 'ВНЕШНЯЯ ПОЧТА:          Если вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:         If the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nКоллеги, доброго дня!\nВысылаю список ориентировочный список вопросов интервью. Если хотим осветить еще какие-то моменты, можем добавить свои.\nВопросы, как я уже сказала, ориентировочные, они лишь задают вектор беседы. Но т.к. все будет проходить в формате живого общения, по факту их может стать больше.\n--\nС уважением,\nКондратьева Мария\nСпециалист программы "Цифровые навыки и компетенции"\nБлаготворительный фонд Сбербанка "Вклад в будущее"\nМоб.тел.: +7(925)197-67-16\n'}

In [183]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[11][1].get('clear_text')

'\nК моему сожалению нет.\n'

In [186]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[11][0]

{'text': 'ВНЕШНЯЯ ПОЧТА:          Если вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:         If the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nДмитрий, добрый день!\nВысылаю телефон Дарии на всякий случай: +7916566279. Если что, звоните ей\nМожете также выслать свой для оперативной связи?\n11:41, 17 октября 2019 г., "Мария Кондратьева" <kondratieva@vbudushee.ru>:\nКоллеги, доброго дня!\nВысылаю список ориентировочный список вопросов интервью. Если хотим осветить еще какие-то моменты, можем добавить свои.\nВопросы, как я уже сказала, ориентировочные, они лишь задают вектор беседы. Но т.к. все будет проходить в формате живого общения, по факту их может стать больше.\n--\nС уважением,\nКондратьева Мария\nСпециа

In [191]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[11]

[{'text': 'ВНЕШНЯЯ ПОЧТА:          Если вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:         If the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nДмитрий, добрый день!\nВысылаю телефон Дарии на всякий случай: +7916566279. Если что, звоните ей\nМожете также выслать свой для оперативной связи?\n11:41, 17 октября 2019 г., "Мария Кондратьева" <kondratieva@vbudushee.ru>:\nКоллеги, доброго дня!\nВысылаю список ориентировочный список вопросов интервью. Если хотим осветить еще какие-то моменты, можем добавить свои.\nВопросы, как я уже сказала, ориентировочные, они лишь задают вектор беседы. Но т.к. все будет проходить в формате живого общения, по факту их может стать больше.\n--\nС уважением,\nКондратьева Мария\nСпеци

In [194]:
list(combine_df[combine_df['Subject_Media']==True].sort_values(by='date')['forward'])[12]

[{'text': 'Добрый день!\nХотелось бы добавить вопрос про тенденции ИИ (общемировые и внутри Банка).\nС уважением,\nКонырев Дмитрий\n'},
 {'to': ['Конырев Дмитрий Андреевич ',
   'Konyrev.D.An@sberbank.ru',
   ' Дария Сатикова ',
   'satikova@vbudushee.ru',
   ' Очеретный Андрей Сергеевич ',
   'Ocheretny.A.S@sberbank.ru',
   ' Мария Кондратьева ',
   'kondratieva@vbudushee.ru'],
  'from': ['Мария Кондратьева ', 'kondratieva@vbudushee.ru'],
  'date': '2019-10-17T17:13:00',
  'subject': 'Re: Выступление на радио Медиаметрикс',
  'text': 'ВНЕШНЯЯ ПОЧТА:\nЕсли вы не знаете отправителя – не открывайте вложений, не переходите по ссылкам, не пересылайте\nконфиденциальную информацию и никогда не вводите свой корпоративный логин и пароль.\nEXTERNAL EMAIL:\nIf the sender is unknown, do not click on links/attachments.\nNever give out confidential information, especially your corporate user ID or password.\n                                \nДмитрий, добрый день!\nВысылаю телефон Дарии на всякий сл

In [215]:
combine_df['subject'].nunique()

1689

### Community по Авто NER

In [211]:

combine_df['Auto_NER'] = np.vectorize(search_pattern)('Community по Авто NER', combine_df['subject'])

In [219]:
for i in range(7):
    r = list(combine_df[combine_df['Auto_NER']==True].sort_values(by='date')['forward'])[i][1].get('clear_text')
    print(r)

Михаил, добрый день!
В Адресбуке не нашел контакты Дмитрия Конорева. Вы сможете выступить на следующей неделе 9 или 10 июля в 12:00? Кто-нибудь кроме тебя и Дмитрия будет?
Спасибо,
Василий
Коллеги, привет!
Идентифицировал Дмитрия 😊
Вам будет удобно выступить 9 или 10 июля в 12:00?
Добрый день!
Не могли бы Вы немного рассказать про community и формат выступления?

Дмитрий, добрый день!
Обычно для community КИБ на встречу выбираем тему для доклада. Участники – D-people КИБ и бизнес заказчики, для которых DS делают модели. В начале выступающий делает презентацию своих материалов (обычно минут 10-30), после этого открытое обсуждение по теме, возможности применения и переиспользования, ответы на вопросы. В среднем приходит около 20 человек + часть сотрудников присоединяются по ВКС из наших ХАБов в других городах.
Предлагаю организовать встречу в среду 10-го июля в 12. Вам подходит?
Спасибо,
Василий
Добрый день,
В целом, 10-е июля удобно. Однако стоит отметить, что AutoNER – это вещь, сущест

In [225]:
for i in range(7):
    from_norm = list(combine_df[combine_df['Auto_NER']==True].sort_values(by='date')['forward'])[i][1].get('from_norm')
    to_norm = list(combine_df[combine_df['Auto_NER']==True].sort_values(by='date')['forward'])[i][1].get('to_norm')
    message_id = list(combine_df[combine_df['Auto_NER']==True].sort_values(by='date')['forward'])[i][1].get('message_id')
    owner_fio = list(combine_df[combine_df['Auto_NER']==True].sort_values(by='date')['forward'])[i][1].get('owner_fio')
    print(from_norm, to_norm, message_id, owner_fio)
    print('\n')

['Потеряев Василий Александрович ', 'poteryaev-va'] ['Кудрявцев Михаил Юрьевич ', 'kudryavtcev-yu'] -5214675080061665055 Poteryaev.V.Al@sberbank.ru


['Потеряев Василий Александрович ', 'poteryaev-va'] ['Кудрявцев Михаил Юрьевич ', 'kudryavtcev-yu', ' Конырев Дмитрий Андреевич ', 'konyrev-da', ' Конырев Дмитрий Андреевич  ', 'konyrev-da'] 9198806269193198668 Конырев Дмитрий Андреевич


['Конырев Дмитрий Андреевич ', 'konyrev-da'] ['Потеряев Василий Александрович ', 'poteryaev-va', ' Кудрявцев Михаил Юрьевич ', 'kudryavtcev-yu', ' Конырев Дмитрий Андреевич  ', 'konyrev-da'] 8175127940749463539 Poteryaev.V.Al@sberbank.ru


['Потеряев Василий Александрович ', 'poteryaev-va'] ['Конырев Дмитрий Андреевич ', 'konyrev-da', ' Кудрявцев Михаил Юрьевич ', 'kudryavtcev-yu', ' Конырев Дмитрий Андреевич  ', 'konyrev-da'] -3329691562811154860 Конырев Дмитрий Андреевич


['Конырев Дмитрий Андреевич ', 'konyrev-da'] ['Потеряев Василий Александрович ', 'poteryaev-va', ' Кудрявцев Михаил Юрьевич ', 'kud

In [244]:
for i in range(40):
    otvet = list(Konyrev_inbox.sort_values(by='date')['forward'])[i][0]
    #from_norm = list(combine_df.sort_values(by='date')['forward'])[i][1].get('from_norm')
    #to_norm = list(combine_df.sort_values(by='date')['forward'])[i][1].get('to_norm')
    #message_id = list(combine_df.sort_values(by='date')['forward'])[i][1].get('message_id')
    #owner_fio = list(combine_df.sort_values(by='date')['forward'])[i][1].get('owner_fio')
    #print(from_norm, to_norm, message_id, owner_fio)
    print(otvet)
    print('\n')

{'text': 'Подтверждено MYUKudryavtcev@sberbank.ru\nОбсуждение дальнейших планов по достижению качества 95%\n'}


{'text': 'FYI\n'}


{'text': 'Дмитрий, добрый день!\nНаправляю\nС уважением,\nИсполнительный директор -\nНачальник Отдела алгоритмизации\nПравового сопровождения кредитного процесса\nПравового департамента\nБурлакова М.В.\n8-557-63-620\n8-910-412-37-01\n'}


{'text': '\n'}


{'text': '\n'}


{'text': 'Дмитрий, спасибо.\nЧитаем, обсуждаем. Комментарии будут чуть позднее.\nПервых два комментария сразу:\n1.       Дорожная карта должна содержать мероприятия по постановке продукта в ПРОМ в мае 2019 года.\n2.       Просьба включить в презентацию текущие результаты ABBYY, ниже таблица.\nТочность извлечения атрибутов\nДокумент\nКол-во док-в\nF-мера\n(по док-м)\nF-мера\n(по атр-м)\nF-макро\nАбсолют\nУстав\n10\n88,57%\n91,33%\n79,32%\n0\nИзм. к Уставу\n10\n77,73%\n80,28%\n62,29%\n0\nПротокол\n10\n89,03%\n89,67%\n73,10%\n1\nТруд. договор\n10\n78,09%\n81,20%\n82,10%\n0\nДС к Труд. догов

TypeError: 'float' object is not subscriptable

In [238]:
list(combine_df.sort_values(by='date')['forward'])[3][0]

{'text': 'Дмитрий, добрый день!\nНаправляю\nС уважением,\nИсполнительный директор -\nНачальник Отдела алгоритмизации\nПравового сопровождения кредитного процесса\nПравового департамента\nБурлакова М.В.\n8-557-63-620\n8-910-412-37-01\n'}

In [254]:
list(Konyrev_inbox.sort_values(by='date')['forward'])[0]

[{'text': 'Подтверждено MYUKudryavtcev@sberbank.ru\nОбсуждение дальнейших планов по достижению качества 95%\n'},
 {'to': [],
  'from': ['Ссылка на событие'],
  'date': '2019-09-01T13:00:00',
  'text': '//rm.sberbank.ru/events/show/1857490>\nСоздано роботом, просьба не отвечать\n',
  'message_id': -5577105929156028601,
  'clear_text': '//rm.sberbank.ru/events/show/1857490>\nСоздано роботом, просьба не отвечать',
  'clear_subject': '',
  'from_norm': ['Ссылка на событие'],
  'to_norm': [],
  'owner': 'MR-admin',
  'from_fio': ['Ссылка на событие'],
  'to_fio': [],
  'owner_fio': 'MR-admin@sberbank.ru'}]

In [326]:
def get_value(forward):
    d = {}
    if forward is None:
        return d
    else:
        if isinstance(forward, list):
            for i in range(len(forward)):
                if  (isinstance(i, int)==True):
                    if (i==0) and (isinstance(forward[i],dict)):
                        d['text'] = forward[i].get('text')
                    if (i==1) and (isinstance(forward[i],dict)):
                        d['clear_text'] = forward[i].get('text')
                        d['from_norm'] = forward[i].get('from_norm')
                        d['to_norm'] = forward[i].get('to_norm')
                        d['from_fio'] = forward[i].get('from_fio')
                        d['to_fio'] = forward[i].get('to_fio')
    return d

In [327]:
get_value(list(Konyrev_inbox.sort_values(by='date')['forward'])[0])

{'text': 'Подтверждено MYUKudryavtcev@sberbank.ru\nОбсуждение дальнейших планов по достижению качества 95%\n',
 'clear_text': '//rm.sberbank.ru/events/show/1857490>\nСоздано роботом, просьба не отвечать\n',
 'from_norm': ['Ссылка на событие'],
 'to_norm': [],
 'from_fio': ['Ссылка на событие'],
 'to_fio': []}

In [273]:
combine_df['dict'] = combine_df['forward'].apply(lambda x : get_value(x))

In [275]:
list(combine_df.sort_values(by='date')['dict'])

[{'text': 'Подтверждено MYUKudryavtcev@sberbank.ru\nОбсуждение дальнейших планов по достижению качества 95%\n',
  'clear_text': '//rm.sberbank.ru/events/show/1857490>\nСоздано роботом, просьба не отвечать\n'},
 {'text': 'FYI\n',
  'clear_text': 'Коллеги, добрый день!\nВ период с 21 по 25 января планируется проведение Операционного комитета по К7М.\nПланируемая повестка Комитета:\n1.  Статус тиражирования К7М.\n2.  Проблемные зоны, пути исправления.\n3.  Активности CDS на 2019: целевая задача  - развитие и поддержка К7М\n4.  Статус выполнения поручения Оперкома № 8 от 24.08.2018: «УРКпИД и ПД разработать MVP продукта с достижением точности извлечения сущностей и атрибутов из документа типа «Устав» не менее 90% в срок до 31.12.2018».\nБольшая просьба подготовить материалы по вопросам, находящимся  в вашей зоне ответственности, и прислать их для свода не позже 16.01.2019.\nС уважением,\nЕ.Иванова\n'},
 {},
 {'text': 'Дмитрий, добрый день!\nНаправляю\nС уважением,\nИсполнительный директор 

In [328]:
combine_df['napical'] = combine_df['forward'].apply(lambda x : get_value(x).get('clear_text'))
combine_df['otvetil'] = combine_df['forward'].apply(lambda x : get_value(x).get('text'))
combine_df['from_norm'] = combine_df['forward'].apply(lambda x : get_value(x).get('from_norm'))
combine_df['to_norm'] = combine_df['forward'].apply(lambda x : get_value(x).get('to_norm'))
combine_df['from_fio'] = combine_df['forward'].apply(lambda x : get_value(x).get('from_fio'))
combine_df['to_fio'] = combine_df['forward'].apply(lambda x : get_value(x).get('to_fio'))

In [329]:
combine_df.head()

,owner,from,to,cc,date,subject,message_id,text,forward,Konyrev,...,Subject_Media,Auto_NER,dict,napical,otvetil,only_me,from_norm,to_norm,from_fio,to_fio
0,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:51:45+00:00,RE: IRPM-202 отсутствует инструкция по установ...,<a1562077a5ff40249595581be759d4f5@omega.sbrf.ru>,"Просьба приложить это в виде инструкции (""рыбу...",[{'text': 'Просьба приложить это в виде инстру...,True,...,False,False,{'text': 'Просьба приложить это в виде инструк...,сопровождения АС ИРД <support ird@mail.ca.sbr...,"Просьба приложить это в виде инструкции (""рыбу...","[0, 0]","[Гапархоев Ваха Лечиевич , gaparkhoev-vl]","[Конырев Дмитрий Андреевич , ЦИД, konyrev-da,...","[Гапархоев Ваха Лечиевич , Gaparkhoev.V.L@omeg...","[Конырев Дмитрий Андреевич , ЦИД, Конырев Дми..."
1,Gaparkhoev.V.L@omega.sbrf.ru,Gaparkhoev.V.L@omega.sbrf.ru,"[Konyrev.D.An@sberbank.ru, Fedotov.I.Ser@omega...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:39:55+00:00,Re: IRPM-202 отсутствует инструкция по установ...,<1577785193.3767.0.camel@omega.sbrf.ru>,"Добрый день, Илья!\r\nМодели и квоты не меняли...","[{'text': 'Добрый день, Илья! Модели и квоты н...",True,...,False,False,"{'text': 'Добрый день, Илья! Модели и квоты не...",; Федотов Илья Сергеевич <Fedotov.I.Ser@omega....,"Добрый день, Илья!\nМодели и квоты не менялись...","[0, 0]","[Гапархоев Ваха Лечиевич , gaparkhoev-vl]","[Конырев Дмитрий Андреевич , ЦИД, konyrev-da]","[Гапархоев Ваха Лечиевич , Gaparkhoev.V.L@omeg...","[Конырев Дмитрий Андреевич , ЦИД, Конырев Дми..."
2,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[AVSipiev@sberbank.ru, Bagnenko.V.V@sberbank.r...",2019-12-31 09:11:08+00:00,IRPM-202 отсутствует инструкция по установке н...,<37eaa2b19f3249bcb211646d4a1a9bdc@omega.sbrf.ru>,"Коллеги, добрый день!\r\n\r\n\r\n\r\nПри устан...","[{'text': 'Коллеги, добрый день! При установке...",True,...,False,False,"{'text': 'Коллеги, добрый день! При установке ...","Добрый день, Илья!\nДа, модели обновились, но ...","Коллеги, добрый день!\nПри установке релиза ht...","[0, 0]","[Гапархоев Ваха Лечиевич , gaparkhoev-vl]","[Конырев Дмитрий Андреевич , ЦИД, konyrev-da,...","[Гапархоев Ваха Лечиевич , Gaparkhoev.V.L@omeg...","[Конырев Дмитрий Андреевич , ЦИД, Конырев Дми..."
3,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-30 14:31:10+00:00,[ACL Portal] Newsletter 2019,<e310dfa11be393364b4403f537f38d52@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,[[ВНЕШНЯЯ ПОЧТА: Если вы не знаете от...,True,...,False,False,{},None,None,[1],None,None,None,None
4,Krivobok.M.A@sberbank.ru,Krivobok.M.A@sberbank.ru,"[Ulyanov.A.A@sberbank.ru, Ocheretny.A.S@sberba...",[AChertok@sberbank.ru],2019-12-30 12:12:04+00:00,RE: материалы со встречи по DS итогам 2019,<67d07ae64f014007b59ab01de9f2c73c@sberbank.ru>,"Коллеги, привет!\r\nОбновлен слайд по TagMe.\r...","[{'text': 'Коллеги, привет! Обновлен слайд по ...",True,...,False,False,"{'text': 'Коллеги, привет! Обновлен слайд по T...","Сведены правки от Льва, Никиты и меня.\nУдалил...","Коллеги, привет!\nОбновлен слайд по TagMe.\nС ...","[0, 0, 0, 0, 0, 0]","[Ульянов Александр Анатольевич , ulyanov-aa]","[Очеретный Андрей Сергеевич , ocheretny-as, З...","[Ульянов Александр Анатольевич , Ulyanov.A.A@s...","[Очеретный Андрей Сергеевич , Ocheretny.A.S@sb..."


In [333]:
combine_df[['from', 'to', 'cc', 'subject', 'napical', 'otvetil', 'date']].to_csv('letters.csv', sep=';', encoding='utf-8')

In [334]:
df = pd.read_csv('letters.csv',sep=';')
df.head()

,Unnamed: 0,from,to,cc,subject,napical,otvetil,date
0,0,Fedotov.I.Ser@omega.sbrf.ru,"['Gaparkhoev.V.L@omega.sbrf.ru', 'Konyrev.D.An...","['Pinchuk-LAL@mail.ca.sbrf.ru', 'support_ird@m...",RE: IRPM-202 отсутствует инструкция по установ...,сопровождения АС ИРД <support ird@mail.ca.sbr...,"Просьба приложить это в виде инструкции (""рыбу...",2019-12-31 09:51:45+00:00
1,1,Gaparkhoev.V.L@omega.sbrf.ru,"['Konyrev.D.An@sberbank.ru', 'Fedotov.I.Ser@om...","['Pinchuk-LAL@mail.ca.sbrf.ru', 'support_ird@m...",Re: IRPM-202 отсутствует инструкция по установ...,; Федотов Илья Сергеевич <Fedotov.I.Ser@omega....,"Добрый день, Илья!\nМодели и квоты не менялись...",2019-12-31 09:39:55+00:00
2,2,Fedotov.I.Ser@omega.sbrf.ru,"['Gaparkhoev.V.L@omega.sbrf.ru', 'Konyrev.D.An...","['AVSipiev@sberbank.ru', 'Bagnenko.V.V@sberban...",IRPM-202 отсутствует инструкция по установке н...,"Добрый день, Илья!\nДа, модели обновились, но ...","Коллеги, добрый день!\nПри установке релиза ht...",2019-12-31 09:11:08+00:00
3,3,portal@aclweb.org,['Konyrev.D.An@sberbank.ru'],[],[ACL Portal] Newsletter 2019,NaN,NaN,2019-12-30 14:31:10+00:00
4,4,Krivobok.M.A@sberbank.ru,"['Ulyanov.A.A@sberbank.ru', 'Ocheretny.A.S@sbe...",['AChertok@sberbank.ru'],RE: материалы со встречи по DS итогам 2019,"Сведены правки от Льва, Никиты и меня.\nУдалил...","Коллеги, привет!\nОбновлен слайд по TagMe.\nС ...",2019-12-30 12:12:04+00:00


### Избавимся от пропусков в данных

In [355]:
len(df) - df.count()

Unnamed: 0    0
from          0
to            0
cc            0
subject       0
napical       0
otvetil       0
date          0
only_me       0
dtype: int64

In [354]:
#заменим Nan на пустую строку, чтобы в дальнейшем избежать ошибок
df['napical'] = df['napical'].astype('str')
df['subject'] = df['subject'].astype('str')
df['otvetil'] = df['otvetil'].astype('str')

### Добавим новые фичи

In [339]:
df.head()

,Unnamed: 0,from,to,cc,subject,napical,otvetil,date,only_me
0,0,Fedotov.I.Ser@omega.sbrf.ru,"['Gaparkhoev.V.L@omega.sbrf.ru', 'Konyrev.D.An...","['Pinchuk-LAL@mail.ca.sbrf.ru', 'support_ird@m...",RE: IRPM-202 отсутствует инструкция по установ...,сопровождения АС ИРД <support ird@mail.ca.sbr...,"Просьба приложить это в виде инструкции (""рыбу...",2019-12-31 09:51:45+00:00,0
1,1,Gaparkhoev.V.L@omega.sbrf.ru,"['Konyrev.D.An@sberbank.ru', 'Fedotov.I.Ser@om...","['Pinchuk-LAL@mail.ca.sbrf.ru', 'support_ird@m...",Re: IRPM-202 отсутствует инструкция по установ...,; Федотов Илья Сергеевич <Fedotov.I.Ser@omega....,"Добрый день, Илья!\nМодели и квоты не менялись...",2019-12-31 09:39:55+00:00,0
2,2,Fedotov.I.Ser@omega.sbrf.ru,"['Gaparkhoev.V.L@omega.sbrf.ru', 'Konyrev.D.An...","['AVSipiev@sberbank.ru', 'Bagnenko.V.V@sberban...",IRPM-202 отсутствует инструкция по установке н...,"Добрый день, Илья!\nДа, модели обновились, но ...","Коллеги, добрый день!\nПри установке релиза ht...",2019-12-31 09:11:08+00:00,0
3,3,portal@aclweb.org,['Konyrev.D.An@sberbank.ru'],[],[ACL Portal] Newsletter 2019,NaN,NaN,2019-12-30 14:31:10+00:00,1
4,4,Krivobok.M.A@sberbank.ru,"['Ulyanov.A.A@sberbank.ru', 'Ocheretny.A.S@sbe...",['AChertok@sberbank.ru'],RE: материалы со встречи по DS итогам 2019,"Сведены правки от Льва, Никиты и меня.\nУдалил...","Коллеги, привет!\nОбновлен слайд по TagMe.\nС ...",2019-12-30 12:12:04+00:00,0


In [317]:
df['to'][2]

"['Gaparkhoev.V.L@omega.sbrf.ru', 'Konyrev.D.An@sberbank.ru']"

In [336]:
def only_me(input_txt):
    pattern = "@[\w]*"
    lst = re.findall(pattern, input_txt)
    if len(lst)==1:
        return 1
    else:
        return 0

In [337]:
only_me(df['to'][2])

0

## письмо адресовано только одному получателю (на поле cc не смотрим)

In [338]:
df['only_me'] = df['to'].apply(lambda x: only_me(x))

In [363]:
df.to_csv('letters.csv', sep=';', encoding='utf-8')

## Если ко мне несколько раз в одном письме общаются по имени, то на такое письмо стоит обратить внимание

In [340]:
def count_my_name(input_txt):
    pattern = "Дмитрий"
    lst = re.findall(pattern, input_txt)
    return len(lst)

In [347]:
df['napical'][1084]

nan

In [356]:
count_my_name(df['napical'][3])

0

In [357]:
df['count_my_name'] = df['napical'].apply(lambda x: count_my_name(x))

In [362]:
list(df[df['count_my_name']>1]['napical'])

['Игорь, добрый день!\nМне сегодня звонил Алексеев Дмитрий, просил передать конфиги по обучению данных. В связи с чем у меня несколько вопросов:\n1.  Что это детально за конфиги\n2.  Разве конфиги не несут никакой информации без кода моделей?\n3.  Если да, то у нас вроде была договоренность по совместному процессу передачи технологий, и если я правильно понимаю, сейчас этот процесс нарушается.\nПросьба, ответить на вопросы.\nЗаранее спасибо!\nС уважением,\nКонырев Дмитрий\n',
 'Коллеги, приветствую\nВладимир,\nИз письма:\nНа встрече мы обсуждали, что применительно к сделкам ДГР возможно понадобиться только доработка моделей в части уставных ограничений. Делать две разные модели (одну текущую, одну под специфичные ограничения) также считаем нецелесообразным, это лишние интеграции между ИРД и моделями, потом свод в единый XML.\nКак Игорь и Дмитрий обсудили на встрече, решение модульное и для решения разных задач будут дорабатываться различные компоненты, ориентируемся на развития в рамка